# Drive verbinden und Imports

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 62.8 MB/s eta 0:00:00


In [4]:
import numpy as np
import rasterio
import geopandas as gpd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Ergebnisse SNAP

In [5]:
# Szenarien und Klassen definieren
szenarien = ["Wohngegend", "Stadt", "Gewerbe", "Land"]
klassen = ["Dach", "Versiegelt"]

# Basispfade
label_base = "/content/drive/MyDrive/Bachelorarbeit/data/Labels/Raster"
result_base = "/content/drive/MyDrive/Bachelorarbeit/results/SNAP"

for sz in szenarien:
    for klasse in klassen:
        # Pfade generieren
        label_path = os.path.join(label_base, f"{sz}_{klasse}.tif")
        result_path = os.path.join(result_base, sz, f"{sz}_{klasse}.tif")

        # Raster laden
        with rasterio.open(label_path) as gt:
            ground_truth = gt.read(1)
        with rasterio.open(result_path) as pred:
            prediction = pred.read(1)

        # IoU berechnen
        intersection = np.logical_and(ground_truth, prediction).sum()
        union = np.logical_or(ground_truth, prediction).sum()
        iou = intersection / union if union != 0 else 0

        # Metriken
        y_true = ground_truth.flatten()
        y_pred = prediction.flatten()
        conf_matrix = confusion_matrix(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred)

        # Ausgabe
        print(f"\n=== {sz} – {klasse} ===")
        print(f"IoU: {round(iou, 3)}")
        print(f"Precision: {round(precision, 3)}")
        print(f"Recall: {round(recall, 3)}")
        print(f"F1-Score: {round(f1, 3)}")
        print(f"Konfusionsmatrix:\n {conf_matrix}")


=== Wohngegend – Dach ===
IoU: 0.827
Precision: 0.918
Recall: 0.893
F1-Score: 0.905
Konfusionsmatrix:
 [[156754   6853]
 [  9266  77127]]

=== Wohngegend – Versiegelt ===
IoU: 0.522
Precision: 0.654
Recall: 0.721
F1-Score: 0.686
Konfusionsmatrix:
 [[186151  17627]
 [ 12885  33337]]

=== Stadt – Dach ===
IoU: 0.84
Precision: 0.871
Recall: 0.959
F1-Score: 0.913
Konfusionsmatrix:
 [[110549  17315]
 [  4994 117142]]

=== Stadt – Versiegelt ===
IoU: 0.749
Precision: 0.917
Recall: 0.804
F1-Score: 0.857
Konfusionsmatrix:
 [[136417   7665]
 [ 20809  85109]]

=== Gewerbe – Dach ===
IoU: 0.947
Precision: 0.949
Recall: 0.998
F1-Score: 0.973
Konfusionsmatrix:
 [[183452   3378]
 [   150  63020]]

=== Gewerbe – Versiegelt ===
IoU: 0.899
Precision: 0.96
Recall: 0.934
F1-Score: 0.947
Konfusionsmatrix:
 [[ 87297   6095]
 [ 10274 146334]]

=== Land – Dach ===
IoU: 0.7
Precision: 0.915
Recall: 0.749
F1-Score: 0.823
Konfusionsmatrix:
 [[196839   3475]
 [ 12494  37192]]

=== Land – Versiegelt ===
IoU: 0.7

# Ergebnisse PolyWorld

In [5]:
# Szenarien definieren
szenarien = ["Wohngegend", "Stadt", "Gewerbe", "Land"]

# Basispfade
label_base = "/content/drive/MyDrive/Bachelorarbeit/data/Labels/Vektor"
polyworld_base = "/content/drive/MyDrive/Bachelorarbeit/results/Polyworld"

for sz in szenarien:
    # Pfade generieren
    label_path = f"{label_base}/{sz}_Dach.gpkg"

    polyworld_file = f"{sz}_PW.gpkg"
    polyworld_path = f"{polyworld_base}/{polyworld_file}"

    # Geodaten einlesen
    gdf_label = gpd.read_file(label_path)
    gdf_polyworld = gpd.read_file(polyworld_path)

    # Geometrien vereinigen
    label_union = gdf_label.geometry.union_all()
    polyworld_union = gdf_polyworld.geometry.union_all()

    # Schnitt- und Vereinigungsfläche berechnen
    intersection_area = label_union.intersection(polyworld_union).area
    union_area = label_union.union(polyworld_union).area

    # IoU berechnen
    iou = intersection_area / union_area

    # Ausgabe
    print(f"IoU (Vektorfläche) – {sz}: {round(iou, 3)}")

IoU (Vektorfläche) – Wohngegend: 0.649
IoU (Vektorfläche) – Stadt: 0.453
IoU (Vektorfläche) – Gewerbe: 0.011
IoU (Vektorfläche) – Land: 0.658


# Ergebnisse SamGeo

In [5]:
szenarien = ["Wohngegend", "Stadt", "Gewerbe", "Land"]
klassen = ["Dach", "Versiegelt"]
base_path = "/content/drive/MyDrive/Bachelorarbeit"
label_raster_base = f"{base_path}/data/Labels/Raster"
label_vector_base = f"{base_path}/data/Labels/Vektor"

## HQSam Point Prompt

In [8]:
hq_path = f"{base_path}/results/SAMGeo/HQ_Sam_point_prompt"

# === Rasterbasierte Auswertung (TIF) ===
print("\n================ Rasterbasierte Metriken (TIF) ================\n")
for sz in szenarien:
    label_path = f"{label_raster_base}/{sz}_Dach.tif"
    pred_path = f"{hq_path}/HQ_pp_{sz}.tif"

    # TIFFs laden
    with rasterio.open(label_path) as src:
        label_raster = src.read(1)
    with rasterio.open(pred_path) as src:
        prediction = src.read(1)

    # Metriken berechnen
    intersection = np.logical_and(label_raster, prediction).sum()
    union = np.logical_or(label_raster, prediction).sum()
    iou = intersection / union if union != 0 else 0

    y_true = label_raster.flatten()
    y_pred = prediction.flatten()
    conf_matrix = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred)

    print(f"--- {sz} ---")
    print(f"IoU:       {round(iou, 3)}")
    print(f"Precision: {round(precision, 3)}")
    print(f"Recall:    {round(recall, 3)}")
    print(f"F1-Score:  {round(f1, 3)}")
    print(f"Konfusionsmatrix:\n{conf_matrix}\n")
# === Vektorbasierte Auswertung (GPKG) ===
print("\n================ Vektorbasierte IoU (GPKG) ================\n")
for sz in szenarien:
    label_path = f"{label_vector_base}/{sz}_Dach.gpkg"
    poly_path = f"{hq_path}/HQ_pp_{sz}.gpkg"
    rect_path = f"{hq_path}/HQ_pp_{sz}_regularized.gpkg"

    gdf_label = gpd.read_file(label_path)
    gdf_poly = gpd.read_file(poly_path)
    gdf_rect = gpd.read_file(rect_path)

    # Sicherstellen, dass CRS übereinstimmen
    crs = gdf_label.crs
    gdf_poly = gdf_poly.to_crs(crs)
    gdf_rect = gdf_rect.to_crs(crs)

    # Vereinigen
    label_union = gdf_label.geometry.union_all()
    poly_union = gdf_poly.geometry.union_all()
    rect_union = gdf_rect.geometry.union_all()

    # IoU berechnen
    iou_poly = label_union.intersection(poly_union).area / label_union.union(poly_union).area
    iou_rect = label_union.intersection(rect_union).area / label_union.union(rect_union).area

    print(f"--- {sz} ---")
    print(f"IoU (normal):      {round(iou_poly, 3)}")
    print(f"IoU (regularized): {round(iou_rect, 3)}\n")


================ Rasterbasierte Metriken (TIF) ================

--- Wohngegend ---
IoU:       0.841
Precision: 0.858
Recall:    0.978
F1-Score:  0.914
Konfusionsmatrix:
[[149587  14020]
 [  1898  84495]]

--- Stadt ---
IoU:       0.599
Precision: 0.601
Recall:    0.994
F1-Score:  0.749
Konfusionsmatrix:
[[ 47260  80604]
 [   743 121393]]

--- Gewerbe ---
IoU:       0.29
Precision: 0.291
Recall:    0.988
F1-Score:  0.45
Konfusionsmatrix:
[[ 34914 151916]
 [   735  62435]]

--- Land ---
IoU:       0.46
Precision: 0.461
Recall:    0.994
F1-Score:  0.63
Konfusionsmatrix:
[[142664  57650]
 [   283  49403]]


================ Vektorbasierte IoU (GPKG) ================

--- Wohngegend ---
IoU (normal):      0.841
IoU (regularized): 0.624

--- Stadt ---
IoU (normal):      0.599
IoU (regularized): 0.489

--- Gewerbe ---
IoU (normal):      0.29
IoU (regularized): 0.253

--- Land ---
IoU (normal):      0.46
IoU (regularized): 0.241



## Sam1 Point Prompt

In [9]:
sam1_path = f"{base_path}/results/SAMGeo/Sam_1_point_prompt"

# === Rasterbasierte Auswertung (TIF) ===
print("\n================ Rasterbasierte Metriken (TIF) – Sam1 ================\n")
for sz in szenarien:
    label_path = f"{label_raster_base}/{sz}_Dach.tif"
    pred_path = f"{sam1_path}/Sam1_pp_{sz}.tif"

    with rasterio.open(label_path) as src:
        label_raster = src.read(1)
    with rasterio.open(pred_path) as src:
        prediction = src.read(1)

    intersection = np.logical_and(label_raster, prediction).sum()
    union = np.logical_or(label_raster, prediction).sum()
    iou = intersection / union if union != 0 else 0

    y_true = label_raster.flatten()
    y_pred = prediction.flatten()
    conf_matrix = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred)

    print(f"--- {sz} ---")
    print(f"IoU:       {round(iou, 3)}")
    print(f"Precision: {round(precision, 3)}")
    print(f"Recall:    {round(recall, 3)}")
    print(f"F1-Score:  {round(f1, 3)}")
    print(f"Konfusionsmatrix:\n{conf_matrix}\n")

# === Vektorbasierte Auswertung (GPKG) ===
print("\n================ Vektorbasierte IoU (GPKG) – Sam1 ================\n")
for sz in szenarien:
    label_path = f"{label_vector_base}/{sz}_Dach.gpkg"
    poly_path = f"{sam1_path}/Sam1_pp_{sz}.gpkg"
    rect_path = f"{sam1_path}/Sam1_pp_{sz}_regularized.gpkg"

    gdf_label = gpd.read_file(label_path)
    gdf_poly = gpd.read_file(poly_path)
    gdf_rect = gpd.read_file(rect_path)

    crs = gdf_label.crs
    gdf_poly = gdf_poly.to_crs(crs)
    gdf_rect = gdf_rect.to_crs(crs)

    label_union = gdf_label.geometry.union_all()
    poly_union = gdf_poly.geometry.union_all()
    rect_union = gdf_rect.geometry.union_all()

    iou_poly = label_union.intersection(poly_union).area / label_union.union(poly_union).area
    iou_rect = label_union.intersection(rect_union).area / label_union.union(rect_union).area

    print(f"--- {sz} ---")
    print(f"IoU (normal):      {round(iou_poly, 3)}")
    print(f"IoU (regularized): {round(iou_rect, 3)}\n")


================ Rasterbasierte Metriken (TIF) – Sam1 ================

--- Wohngegend ---
IoU:       0.807
Precision: 0.864
Recall:    0.924
F1-Score:  0.893
Konfusionsmatrix:
[[151073  12534]
 [  6579  79814]]

--- Stadt ---
IoU:       0.597
Precision: 0.626
Recall:    0.927
F1-Score:  0.747
Konfusionsmatrix:
[[ 60243  67621]
 [  8921 113215]]

--- Gewerbe ---
IoU:       0.25
Precision: 0.251
Recall:    0.985
F1-Score:  0.4
Konfusionsmatrix:
[[   877 185953]
 [   924  62246]]

--- Land ---
IoU:       0.455
Precision: 0.456
Recall:    0.993
F1-Score:  0.625
Konfusionsmatrix:
[[141466  58848]
 [   357  49329]]


================ Vektorbasierte IoU (GPKG) – Sam1 ================

--- Wohngegend ---
IoU (normal):      0.806
IoU (regularized): 0.645

--- Stadt ---
IoU (normal):      0.597
IoU (regularized): 0.489

--- Gewerbe ---
IoU (normal):      0.25
IoU (regularized): 0.253

--- Land ---
IoU (normal):      0.454
IoU (regularized): 0.286



## Sam2 Point Prompt

In [10]:
# === Modellvarianten von Sam2 ===
sam2_varianten = {
    "Sam2_500": f"{base_path}/results/SAMGeo/Sam_2_point_prompt/minsize500_thresh2000",
    "Sam2_1000": f"{base_path}/results/SAMGeo/Sam_2_point_prompt/mimsize1000_thresh2000"
}

# === Auswertung für jede Variante ===
for modellname, modellpfad in sam2_varianten.items():
    size = modellname.split("_")[1]  # "500" oder "1000"
    print(f"\n================ Rasterbasierte Metriken (TIF) – {modellname} ================\n")

    for sz in szenarien:
        label_path = f"{label_raster_base}/{sz}_Dach.tif"
        pred_path = os.path.join(modellpfad, f"Sam2_pp_{size}_{sz}.tif")

        with rasterio.open(label_path) as src:
            label_raster = src.read(1)
        with rasterio.open(pred_path) as src:
            prediction = src.read(1)

        intersection = np.logical_and(label_raster, prediction).sum()
        union = np.logical_or(label_raster, prediction).sum()
        iou = intersection / union if union != 0 else 0

        y_true = label_raster.flatten()
        y_pred = prediction.flatten()
        conf_matrix = confusion_matrix(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred)

        print(f"--- {sz} ---")
        print(f"IoU:       {round(iou, 3)}")
        print(f"Precision: {round(precision, 3)}")
        print(f"Recall:    {round(recall, 3)}")
        print(f"F1-Score:  {round(f1, 3)}")
        print(f"Konfusionsmatrix:\n{conf_matrix}\n")

    print(f"\n================ Vektorbasierte IoU (GPKG) – {modellname} ================\n")
    for sz in szenarien:
        label_path = f"{label_vector_base}/{sz}_Dach.gpkg"
        poly_path = os.path.join(modellpfad, f"Sam2_pp_{size}_{sz}.gpkg")
        rect_path = os.path.join(modellpfad, f"Sam2_pp_{size}_{sz}_regularized.gpkg")

        gdf_label = gpd.read_file(label_path)
        gdf_poly = gpd.read_file(poly_path)
        gdf_rect = gpd.read_file(rect_path)

        crs = gdf_label.crs
        gdf_poly = gdf_poly.to_crs(crs)
        gdf_rect = gdf_rect.to_crs(crs)

        label_union = gdf_label.geometry.union_all()
        poly_union = gdf_poly.geometry.union_all()
        rect_union = gdf_rect.geometry.union_all()

        iou_poly = label_union.intersection(poly_union).area / label_union.union(poly_union).area
        iou_rect = label_union.intersection(rect_union).area / label_union.union(rect_union).area

        print(f"--- {sz} ---")
        print(f"IoU (normal):      {round(iou_poly, 3)}")
        print(f"IoU (regularized): {round(iou_rect, 3)}\n")


================ Rasterbasierte Metriken (TIF) – Sam2_500 ================

--- Wohngegend ---
IoU:       0.874
Precision: 0.92
Recall:    0.946
F1-Score:  0.933
Konfusionsmatrix:
[[156535   7072]
 [  4694  81699]]

--- Stadt ---
IoU:       0.822
Precision: 0.912
Recall:    0.893
F1-Score:  0.902
Konfusionsmatrix:
[[117302  10562]
 [ 13033 109103]]

--- Gewerbe ---
IoU:       0.892
Precision: 0.963
Recall:    0.924
F1-Score:  0.943
Konfusionsmatrix:
[[184578   2252]
 [  4790  58380]]

--- Land ---
IoU:       0.734
Precision: 0.746
Recall:    0.979
F1-Score:  0.847
Konfusionsmatrix:
[[183716  16598]
 [  1021  48665]]


================ Vektorbasierte IoU (GPKG) – Sam2_500 ================

--- Wohngegend ---
IoU (normal):      0.873
IoU (regularized): 0.757

--- Stadt ---
IoU (normal):      0.822
IoU (regularized): 0.758

--- Gewerbe ---
IoU (normal):      0.892
IoU (regularized): 0.823

--- Land ---
IoU (normal):      0.733
IoU (regularized): 0.642


================ Rasterbasierte Me

## Sam1 Text Prompt

In [6]:
modellname = "Sam1_tp"
modellpfad = f"{base_path}/results/SAMGeo/Sam1_text_prompt"

# === Rastermetriken ===
print(f"\n================ Rastermetriken (TIF) – {modellname} ================\n")
for sz in szenarien:
    for klasse in klassen:
        label_path = os.path.join(label_raster_base, f"{sz}_{klasse}.tif")
        if klasse == "Dach":
            pred_path = os.path.join(modellpfad, f"{modellname}_{sz}.tif")
        else:
            pred_path = os.path.join(modellpfad, f"{modellname}_{sz}_{klasse}.tif")

        with rasterio.open(label_path) as src:
            label_raster = src.read(1)

        with rasterio.open(pred_path) as src:
            prediction = src.read(1)
        prediction = (prediction == 255).astype(np.uint8)

        intersection = np.logical_and(label_raster, prediction).sum()
        union = np.logical_or(label_raster, prediction).sum()
        iou = intersection / union if union != 0 else 0

        y_true = label_raster.flatten()
        y_pred = prediction.flatten()
        conf_matrix = confusion_matrix(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred)

        print(f"--- {sz} – {klasse} ---")
        print(f"IoU:       {round(iou, 3)}")
        print(f"Precision: {round(precision, 3)}")
        print(f"Recall:    {round(recall, 3)}")
        print(f"F1-Score:  {round(f1, 3)}")
        print(f"Konfusionsmatrix:\n{conf_matrix}\n")

# === Vektor-IoU ===
print(f"\n================ Vektor-IoU (GPKG) – {modellname} ================\n")
for sz in szenarien:
    for klasse in klassen:
        label_path = os.path.join(label_vector_base, f"{sz}_{klasse}.gpkg")
        if klasse == "Dach":
            poly_path = os.path.join(modellpfad, f"{modellname}_{sz}.gpkg")
            rect_path = os.path.join(modellpfad, f"{modellname}_{sz}_regularized.gpkg")
        else:
            poly_path = os.path.join(modellpfad, f"{modellname}_{sz}_{klasse}.gpkg")
            rect_path = None  # keine regularized-Version bei Versiegelt

        gdf_label = gpd.read_file(label_path)
        gdf_poly = gpd.read_file(poly_path)

        crs = gdf_label.crs
        gdf_poly = gdf_poly.to_crs(crs)

        label_union = gdf_label.geometry.union_all()
        poly_union = gdf_poly.geometry.union_all()
        iou_poly = label_union.intersection(poly_union).area / label_union.union(poly_union).area

        print(f"--- {sz} – {klasse} ---")
        print(f"IoU (normal):      {round(iou_poly, 3)}")

        if klasse == "Dach" and rect_path and os.path.exists(rect_path):
            gdf_rect = gpd.read_file(rect_path).to_crs(crs)
            rect_union = gdf_rect.geometry.union_all()
            iou_rect = label_union.intersection(rect_union).area / label_union.union(rect_union).area
            print(f"IoU (regularized): {round(iou_rect, 3)}")

        print()


================ Rastermetriken (TIF) – Sam1_tp ================

--- Wohngegend – Dach ---
IoU:       0.534
Precision: 0.545
Recall:    0.964
F1-Score:  0.696
Konfusionsmatrix:
[[94073 69534]
 [ 3090 83303]]

--- Wohngegend – Versiegelt ---
IoU:       0.261
Precision: 0.346
Recall:    0.515
F1-Score:  0.414
Konfusionsmatrix:
[[158861  44917]
 [ 22431  23791]]

--- Stadt – Dach ---
IoU:       0.557
Precision: 0.605
Recall:    0.876
F1-Score:  0.716
Konfusionsmatrix:
[[ 57983  69881]
 [ 15149 106987]]

--- Stadt – Versiegelt ---
IoU:       0.372
Precision: 0.501
Recall:    0.592
F1-Score:  0.543
Konfusionsmatrix:
[[81636 62446]
 [43239 62679]]

--- Gewerbe – Dach ---
IoU:       0.291
Precision: 0.298
Recall:    0.927
F1-Score:  0.451
Konfusionsmatrix:
[[ 49051 137779]
 [  4636  58534]]

--- Gewerbe – Versiegelt ---
IoU:       0.59
Precision: 0.702
Recall:    0.787
F1-Score:  0.742
Konfusionsmatrix:
[[ 40993  52399]
 [ 33391 123217]]

--- Land – Dach ---
IoU:       0.766
Precision: 0.84

## Sam2 Text Prompt

In [7]:
modellname = "Sam2_tp"
modellpfad = f"{base_path}/results/SAMGeo/Sam2_text_prompt"

# === Rastermetriken ===
print(f"\n================ Rastermetriken (TIF) – {modellname} ================\n")
for sz in szenarien:
    for klasse in klassen:
        label_path = os.path.join(label_raster_base, f"{sz}_{klasse}.tif")
        if klasse == "Dach":
            pred_path = os.path.join(modellpfad, f"{modellname}_{sz}.tif")
        else:
            pred_path = os.path.join(modellpfad, f"{modellname}_{sz}_{klasse}.tif")

        with rasterio.open(label_path) as src:
            label_raster = src.read(1)

        with rasterio.open(pred_path) as src:
            prediction = src.read(1)
        prediction = (prediction == 255).astype(np.uint8)

        intersection = np.logical_and(label_raster, prediction).sum()
        union = np.logical_or(label_raster, prediction).sum()
        iou = intersection / union if union != 0 else 0

        y_true = label_raster.flatten()
        y_pred = prediction.flatten()
        conf_matrix = confusion_matrix(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred)

        print(f"--- {sz} – {klasse} ---")
        print(f"IoU:       {round(iou, 3)}")
        print(f"Precision: {round(precision, 3)}")
        print(f"Recall:    {round(recall, 3)}")
        print(f"F1-Score:  {round(f1, 3)}")
        print(f"Konfusionsmatrix:\n{conf_matrix}\n")

# === Vektor-IoU ===
print(f"\n================ Vektor-IoU (GPKG) – {modellname} ================\n")
for sz in szenarien:
    for klasse in klassen:
        label_path = os.path.join(label_vector_base, f"{sz}_{klasse}.gpkg")
        if klasse == "Dach":
            poly_path = os.path.join(modellpfad, f"{modellname}_{sz}.gpkg")
            rect_path = os.path.join(modellpfad, f"{modellname}_{sz}_regularized.gpkg")
        else:
            poly_path = os.path.join(modellpfad, f"{modellname}_{sz}_{klasse}.gpkg")
            rect_path = None  # keine regularized-Version bei Versiegelt

        gdf_label = gpd.read_file(label_path)
        gdf_poly = gpd.read_file(poly_path)

        crs = gdf_label.crs
        gdf_poly = gdf_poly.to_crs(crs)

        label_union = gdf_label.geometry.union_all()
        poly_union = gdf_poly.geometry.union_all()
        iou_poly = label_union.intersection(poly_union).area / label_union.union(poly_union).area

        print(f"--- {sz} – {klasse} ---")
        print(f"IoU (normal):      {round(iou_poly, 3)}")

        if klasse == "Dach" and rect_path and os.path.exists(rect_path):
            gdf_rect = gpd.read_file(rect_path).to_crs(crs)
            rect_union = gdf_rect.geometry.union_all()
            iou_rect = label_union.intersection(rect_union).area / label_union.union(rect_union).area
            print(f"IoU (regularized): {round(iou_rect, 3)}")

        print()


================ Rastermetriken (TIF) – Sam2_tp ================

--- Wohngegend – Dach ---
IoU:       0.853
Precision: 0.884
Recall:    0.961
F1-Score:  0.921
Konfusionsmatrix:
[[152682  10925]
 [  3333  83060]]

--- Wohngegend – Versiegelt ---
IoU:       0.205
Precision: 0.999
Recall:    0.205
F1-Score:  0.34
Konfusionsmatrix:
[[203768     10]
 [ 36737   9485]]

--- Stadt – Dach ---
IoU:       0.627
Precision: 0.7
Recall:    0.857
F1-Score:  0.771
Konfusionsmatrix:
[[ 82918  44946]
 [ 17411 104725]]

--- Stadt – Versiegelt ---
IoU:       0.287
Precision: 0.904
Recall:    0.296
F1-Score:  0.446
Konfusionsmatrix:
[[140740   3342]
 [ 74556  31362]]

--- Gewerbe – Dach ---
IoU:       0.387
Precision: 0.387
Recall:    1.0
F1-Score:  0.558
Konfusionsmatrix:
[[87009 99821]
 [   25 63145]]

--- Gewerbe – Versiegelt ---
IoU:       0.463
Precision: 0.644
Recall:    0.623
F1-Score:  0.633
Konfusionsmatrix:
[[39449 53943]
 [59032 97576]]

--- Land – Dach ---
IoU:       0.746
Precision: 0.825
Re